In [1]:
import numpy as np

# ============ 1. 生成一些简单的假数据 ============
# 这里用随机数来模拟输入数据 X 和标签 y
# X 维度为 (N, 2)，其中 N 是样本数，每个样本有 2 个特征
# y 维度为 (N, 1)，假设做一个简单的回归或二分类（0或1）
np.random.seed(42)  # 固定随机种子，方便复现
N = 100   # 样本数
X = np.random.rand(N, 2)  # 生成随机输入
# 生成一个简单的目标函数：y = x1 + x2 + 一点噪声
y = X[:, [0]] + X[:, [1]] + 0.1 * np.random.randn(N, 1)

# ============ 2. 初始化网络参数 ============
# 假设单隐藏层含有 hidden_dim=3 个神经元
input_dim = 2
hidden_dim = 3
output_dim = 1

# 权重初始化可以用小随机数
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)  # shape = [2, 3]
b1 = np.zeros((1, hidden_dim))                     # shape = [1, 3]

W2 = 0.1 * np.random.randn(hidden_dim, output_dim) # shape = [3, 1]
b2 = np.zeros((1, output_dim))                     # shape = [1, 1]

# ============ 3. 定义辅助函数（激活函数、损失函数等） ============

# 1) 激活函数，这里使用 ReLU；也可以使用 sigmoid 或 tanh
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU 的导数：x>0 时为1，x<=0 时为0
    return (x > 0).astype(float)

# 2) 损失函数，这里以均方误差(MSE)为例
def mean_squared_error(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

# ============ 4. 设置训练超参数 ============
learning_rate = 0.01
num_epochs = 2000  # 训练轮数

# 为了观察训练过程中的损失走向，保存每一次迭代的loss
loss_history = []

# ============ 5. 训练循环 ============
for epoch in range(num_epochs):
    # ---- (1) 前向传播 (Forward Pass) ----
    # a1 = ReLU(X * W1 + b1)
    z1 = np.dot(X, W1) + b1  # shape: (N, 3)
    a1 = relu(z1)
    
    # y_pred = a1 * W2 + b2； 这里假设输出层不加激活函数(做回归)
    z2 = np.dot(a1, W2) + b2  # shape: (N, 1)
    y_pred = z2  # 对回归任务，最后一层不加激活

    # ---- (2) 计算损失 ----
    loss = mean_squared_error(y_pred, y)
    loss_history.append(loss)

    # ---- (3) 反向传播 (Backpropagation) ----
    # ① 计算输出层对 z2 的梯度 dL/dz2
    # 对 MSE 来说，dL/dy_pred = (y_pred - y)，而 y_pred=z2，因此 dL/dz2 = 1/N * 2*(z2 - y) 
    # （不过常常省略常数因子 2 或者 1/N，和学习率一起调，不影响收敛）
    dL_dz2 = (y_pred - y)  # shape: (N, 1)

    # ② 计算对 W2 和 b2 的梯度
    # dL/dW2 = a1^T * dL/dz2
    dW2 = np.dot(a1.T, dL_dz2) / N  # 平均
    db2 = np.sum(dL_dz2, axis=0, keepdims=True) / N

    # ③ 传播到隐藏层：dL/da1 = dL/dz2 * dz2/da1
    # 但 z2 = a1 * W2 + b2，只要按矩阵乘法反向即可
    dL_da1 = np.dot(dL_dz2, W2.T)  # shape: (N, 3)

    # ④ 计算对 z1 的梯度 dL/dz1 (考虑 ReLU 的导数)
    dL_dz1 = dL_da1 * relu_deriv(z1)  # shape: (N, 3)

    # ⑤ 计算对 W1 和 b1 的梯度
    dW1 = np.dot(X.T, dL_dz1) / N  # 平均
    db1 = np.sum(dL_dz1, axis=0, keepdims=True) / N

    # ---- (4) 参数更新 (Gradient Descent) ----
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # ---- (5) 打印训练过程 ----
    if (epoch+1) % 200 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss = {loss:.6f}")

# ============ 6. 训练完成后的结果 ============
print("\n训练完成后最终Loss: {:.6f}".format(loss_history[-1]))

# 这里我们可以查看任意几条样本的预测输出与真实值对比
test_indices = [0, 1, 2, 3, 4]
print("\n部分预测结果:")
for i in test_indices:
    print(f"X={X[i]}, y_true={y[i]}, y_pred={y_pred[i]}")


Epoch [200/2000], Loss = 0.168021
Epoch [400/2000], Loss = 0.145609
Epoch [600/2000], Loss = 0.130411
Epoch [800/2000], Loss = 0.112941
Epoch [1000/2000], Loss = 0.094166
Epoch [1200/2000], Loss = 0.075462
Epoch [1400/2000], Loss = 0.058259
Epoch [1600/2000], Loss = 0.043693
Epoch [1800/2000], Loss = 0.032418
Epoch [2000/2000], Loss = 0.024340

训练完成后最终Loss: 0.024340

部分预测结果:
X=[0.37454012 0.95071431], y_true=[1.25725195], y_pred=[1.23120023]
X=[0.73199394 0.59865848], y_true=[1.3538778], y_pred=[1.27301047]
X=[0.15601864 0.15599452], y_true=[0.34132041], y_pred=[0.50659318]
X=[0.05808361 0.86617615], y_true=[0.85282462], y_pred=[0.91993813]
X=[0.60111501 0.70807258], y_true=[1.49576504], y_pred=[1.24421276]
